In [ ]:
%pip install vosk
%pip install pydub
%pip install transformers
%pip install ipywidgets

In [ ]:
import json
from vosk import Model, KaldiRecognizer
from pydub import AudioSegment

FRAME_RATE = 16000
CHANNELS = 1

def voice_recognition(filename):
  model = Model(model_name="vosk-model-pt-fb-v0.1.1-20220516_2113")
  rec = KaldiRecognizer(model, FRAME_RATE)
  rec.SetWords(True)

  wav = AudioSegment.from_wav(filename)
  wav = wav.set_channels(CHANNELS)
  wav = wav.set_frame_rate(FRAME_RATE)

  step = 45000
  transcript = ""
  for i in range(0, len(wav), step):
    print(f"Progress: {i/len(wav)}")
    segment = wav[i:(i+step)]

    rec.AcceptWaveform(segment.raw_data)
    result = rec.Result()

    text = json.loads(result)["text"]
    transcript += text

  return transcript

In [ ]:
import ipywidgets as widgets

uploader = widgets.FileUpload()
display(uploader)

FileUpload(value={}, description='Upload')

In [ ]:
from pydub import AudioSegment
import tempfile
import os

def process_uploaded_file(change):
    if uploader.value:
        up_file = uploader.value[list(uploader.value.keys())[0]]
        with tempfile.NamedTemporaryFile(delete=False) as temp_audio:
            temp_audio.write(up_file["content"])
            temp_audio_path = temp_audio.name

        transcript = voice_recognition(temp_audio_path)
        print("Transcript:", transcript)

        # Clean up the temporary audio file
        os.remove(temp_audio_path)

        return transcript

uploader.observe(process_uploaded_file, names='value')

process_uploaded_file(uploader.value)

Progress: 0.0
Transcript: o atacante mario já cancelei a aula de amanhã e aí agora se segue seu horário que você você é vinte e oito se não fizer duas aulas de moto né aí conclui moto dia trinta e um se faz uma aula de carro e dia quatro de setembro seu faz a última aula de carro


'o atacante mario já cancelei a aula de amanhã e aí agora se segue seu horário que você você é vinte e oito se não fizer duas aulas de moto né aí conclui moto dia trinta e um se faz uma aula de carro e dia quatro de setembro seu faz a última aula de carro'

In [ ]:
# Summering the audio

In [ ]:
from transformers import pipeline
summarizer = pipeline("summarization")

transcript = process_uploaded_file(uploader.value)

split_tokens = transcript.split(" ")
docs = []

for i in range(0, len(split_tokens), 850):
  selection = " ".join(split_tokens[i:(i+850)])
  docs.append(selection)

summaries = summarizer(docs)

summary = "\n\n".join([d["summary_text"] for d in summaries])

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Progress: 0.0
Transcript: o atacante mario já cancelei a aula de amanhã e aí agora se segue seu horário que veio pra você é vinte e oito se não fizer duas aulas de moto né aí conclui moto dia trinta e um se faz uma aula de carro e dia quatro de setembro seu faz a última aula de carro


Your max_length is set to 142, but your input_length is only 108. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=54)


In [ ]:
print(summary)

 Você é vinte e oito se não fizer duas aulas de moto né aí conclui moto dia trinta e um se faz uma aula de carro e dia quatro de setembro seu faz aula of amanhã .
